# Tree-based Models, with Clusters

## Library Imports

In [1]:
# Necessary code to import our helper functions
import sys
sys.path.append("../..")

In [2]:
# Library imports
import numpy as np
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from matplotlib import pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from Common_Functions import data_split, add_unique_identifier, data_cleaning, hospital_data_agg, optimal_k

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
# Method from Shruti's code
def standardize_data(train_data, val_data):
    train_temp = train_data.drop(columns = ['site','cluster','lat','lon'])
    val_temp = val_data.drop(columns = ['site','cluster','lat','lon'])
    
    scaler = MinMaxScaler()
    
    train_data_scaled = scaler.fit_transform(train_temp)
    train_data_scaled = pd.DataFrame(train_data_scaled, columns = train_temp.columns)
    train_data_scaled['cluster'] = train_data['cluster'].to_list()
    train_data_scaled['site'] = train_data['site'].to_list()
    train_data_scaled['lat'] = train_data['lat'].to_list()
    train_data_scaled['lon'] = train_data['lon'].to_list()
    
    val_data_scaled = scaler.transform(val_temp)
    val_data_scaled = pd.DataFrame(val_data_scaled, columns = val_temp.columns)
    val_data_scaled['cluster'] = val_data['cluster'].to_list()
    val_data_scaled['site'] = val_data['site'].to_list()
    val_data_scaled['lat'] = val_data['lat'].to_list()
    val_data_scaled['lon'] = val_data['lon'].to_list()
    
    return train_data_scaled, val_data_scaled

In [4]:
# Method slightly modified from Shruti's code
def impute_knn(train_data, val_data, optimal_k): 
    train_data_scaled, val_data_scaled = standardize_data(train_data, val_data)

    knn = KNNImputer(n_neighbors = optimal_k)

    # imputing values
    train_data_scaled[list(train_data_scaled.columns)] = knn.fit_transform(train_data_scaled)
    val_data_scaled[list(val_data_scaled.columns)] = knn.transform(val_data_scaled)
    
    return train_data_scaled, val_data_scaled

## Data Import

In [5]:
data = pd.read_csv("../../Feature Matrix/processed_data.csv")
data.dropna(subset = ['mcare_count'], inplace = True)
data.drop(columns = ['year'], inplace = True)
data.replace([np.inf, -np.inf], 0, inplace=True)

## Model Parameters

In [6]:
COUNT_THRESH = 34
RDM_SEED = 123
TRAIN_TEST_PROPORTION = 0.8

## Data Transformation

### One-Hot Categorical Encoding and Dropping NAs

In [7]:
data = data_cleaning(data, dropna = False, one_hot = False)

### Data Split

In [8]:
working_set, predict_set = data_split(data, count_thresh = COUNT_THRESH)

In [9]:
model_data = working_set
predict_data = predict_set

In [10]:
display(model_data)

,site,group,priv_count,priv_pay_median,mcare_los,mcare_pay_median,CBSA_NAME,lon,lat,Hospitals,...,annual_births,frac_veteran,frac_disability,non_citizen,employment_rate,frac_priv_insurance,frac_mcare_insurance,frac_no_insurance,cluster,mcare_count
40,1,breast reconstruction,63,24289.900,2.549296,8794.190,"Dallas-Fort Worth-Arlington, TX",-96.920913,32.707875,114.0,...,1974825.0,0.06,0.10,0.59,0.69,0.66,0.25,0.17,0,71.0
70,1,breast reconstruction,51,21408.000,3.543210,10395.160,"Houston-The Woodlands-Sugar Land, TX",-95.622552,29.598443,181.0,...,1808878.0,0.05,0.10,0.57,0.66,0.60,0.28,0.19,0,81.0
112,1,breast reconstruction,64,29757.100,3.918699,14174.100,"New York-Newark-Jersey City, NY-NJ-PA",-74.005954,40.712776,143.0,...,4668590.0,0.03,0.11,0.41,0.65,0.66,0.39,0.07,0,123.0
219,1,breast reconstruction,66,25240.905,3.241935,10144.445,"Dallas-Fort Worth-Arlington, TX",-96.920913,32.707875,114.0,...,1974825.0,0.06,0.10,0.59,0.69,0.66,0.25,0.17,0,62.0
275,1,breast reconstruction,45,34963.900,3.262295,14008.190,"New York-Newark-Jersey City, NY-NJ-PA",-74.005954,40.712776,143.0,...,4668590.0,0.03,0.11,0.41,0.65,0.66,0.39,0.07,0,122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44974,0,ant_cerv_fusion,54,17137.490,NaN,6934.483,"Atlanta-Sandy Springs-Alpharetta, GA",-84.294090,34.075380,80.0,...,1573561.0,0.07,0.11,0.48,0.67,0.70,0.28,0.12,2,0.0
44991,0,ant_cerv_fusion,39,11807.250,NaN,6905.252,"Charlotte-Concord-Gastonia, NC-SC",-80.721440,35.122320,26.0,...,673803.0,0.06,0.11,0.57,0.67,0.69,0.31,0.10,2,0.0
45038,0,ant_cerv_fusion,35,18421.250,NaN,6724.500,"Memphis, TN-MS-AR",-89.850500,35.038720,28.0,...,327202.0,0.07,0.13,0.64,0.64,0.64,0.37,0.11,2,0.0
45100,0,ant_cerv_fusion,51,13926.250,NaN,7334.787,"Atlanta-Sandy Springs-Alpharetta, GA",-84.294090,34.075380,80.0,...,1573561.0,0.07,0.11,0.48,0.67,0.70,0.28,0.12,2,0.0


## Dev/Test Split

In [11]:
X_input = model_data.drop(columns=["priv_pay_median"])
y_input = model_data["priv_pay_median"]

# display(X_input)
# display(y_input)

X_dev, X_test, y_dev, y_test = train_test_split(X_input,
                                                y_input,
                                                train_size = TRAIN_TEST_PROPORTION,
                                                random_state = RDM_SEED)
display(X_dev)

,site,group,priv_count,mcare_los,mcare_pay_median,CBSA_NAME,lon,lat,Hospitals,PctTeaching,...,annual_births,frac_veteran,frac_disability,non_citizen,employment_rate,frac_priv_insurance,frac_mcare_insurance,frac_no_insurance,cluster,mcare_count
22077,0,fess,69,0.000000,3698.19,"Muskegon, MI",-86.248392,43.234181,5.0,0.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,81.0
7644,0,rtc_slap_bank,50,0.000000,4341.63,"Jackson, MI",-84.401346,42.245869,2.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,99.0
16125,1,tka,230,1.800551,10385.95,"Houston-The Woodlands-Sugar Land, TX",-95.622552,29.598443,181.0,0.088398,...,1808878.0,0.05,0.10,0.57,0.66,0.60,0.28,0.19,2,1815.0
31370,0,hernia,44,0.000000,2132.26,"Memphis, TN-MS-AR",-89.850500,35.038720,28.0,0.357143,...,327202.0,0.07,0.13,0.64,0.64,0.64,0.37,0.11,2,189.0
13308,1,tha,62,2.970817,10201.24,"Birmingham-Hoover, AL",-86.811378,33.405387,24.0,0.250000,...,264679.0,0.06,0.15,0.58,0.61,0.69,0.35,0.08,2,1028.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13859,1,tha,47,3.479332,15409.19,"Seattle-Bellevue-Everett, WA",-122.195948,47.607680,38.0,0.368421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1137.0
33698,0,hysterect,223,0.000000,7867.29,"Portland-Vancouver-Hillsboro, OR-WA",-122.546300,45.653680,30.0,0.366667,...,617728.0,0.07,0.13,0.43,0.66,0.72,0.35,0.06,0,203.0
12634,0,radius/ulna internal fixation,37,0.000000,4160.69,"Virginia Beach-Norfolk-Newport News, VA-NC",-76.285873,36.850769,20.0,0.500000,...,434583.0,0.15,0.14,0.38,0.67,0.74,0.36,0.06,0,141.0
30258,1,hernia,48,5.243728,10176.57,"Atlanta-Sandy Springs-Alpharetta, GA",-84.294090,34.075380,80.0,0.162500,...,1573561.0,0.07,0.11,0.48,0.67,0.70,0.28,0.12,2,279.0


## Target Encoding

In [12]:
# Target encoding group variable
te_group = TargetEncoder()
X_dev['group_encoded'] = te_group.fit_transform(X_dev['group'],y_dev)
X_dev.drop(columns = 'group', inplace = True)
X_test['group_encoded'] = te_group.transform(X_test['group'])
X_test.drop(columns = 'group', inplace = True)

# Target encoding CBSA Name
te_CBSA_NAME = TargetEncoder()
X_dev['CBSA_NAME_encoded'] = te_CBSA_NAME.fit_transform(X_dev['CBSA_NAME'],y_dev)
X_dev.drop(columns = 'CBSA_NAME', inplace = True)
X_test['CBSA_NAME_encoded'] = te_CBSA_NAME.transform(X_test['CBSA_NAME'])
X_test.drop(columns = 'CBSA_NAME', inplace = True)

C:\Users\rogmo\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\rogmo\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
C:\Users\rogmo\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\rogmo\anaconda3\lib\site-packages\category_encoders\target_enco

## KNN Imputation

In [13]:
knn_data = X_dev.copy()
knn_data['priv_pay_median'] = y_dev

# optimal_k = optimal_k(knn_data)
# print(optimal_k)
optimal_k = 2

X_dev_imputed, X_test_imputed = impute_knn(X_dev, X_test, optimal_k)
display(X_dev_imputed)

,priv_count,mcare_los,mcare_pay_median,Hospitals,PctTeaching,PctLargeHospital,PctPrivate,total_population,median_age,sex_ratio,...,frac_priv_insurance,frac_mcare_insurance,frac_no_insurance,mcare_count,group_encoded,CBSA_NAME_encoded,cluster,site,lat,lon
0,0.018231,0.000000,0.072701,0.019704,0.400000,0.000000,0.400000,0.004727,0.518657,0.422460,...,0.717391,0.787879,0.107143,0.009075,0.098144,0.195468,0.0,0.0,43.234181,-86.248392
1,0.008043,0.000000,0.085350,0.004926,0.000000,0.000000,0.000000,0.003891,0.559701,0.828877,...,0.717391,0.787879,0.107143,0.011091,0.117025,0.174497,0.0,0.0,42.245869,-84.401346
2,0.104558,0.161594,0.204171,0.886700,0.088398,0.182320,0.823204,0.361869,0.350746,0.497326,...,0.521739,0.333333,0.571429,0.203339,0.297375,0.627942,2.0,1.0,29.598443,-95.622552
3,0.004826,0.000000,0.041917,0.133005,0.357143,0.428571,0.571429,0.063652,0.399254,0.112299,...,0.608696,0.606061,0.285714,0.021174,0.092131,0.311718,2.0,0.0,35.038720,-89.850500
4,0.014477,0.266622,0.200540,0.113300,0.250000,0.500000,0.583333,0.052365,0.503731,0.208556,...,0.717391,0.545455,0.178571,0.115169,0.326175,0.182142,2.0,1.0,33.405387,-86.811378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3736,0.006434,0.312259,0.302920,0.182266,0.368421,0.078947,0.421053,0.123304,0.492537,0.529412,...,0.782609,0.545455,0.107143,0.127381,0.326175,0.822189,2.0,1.0,47.607680,-122.195948
3737,0.100804,0.000000,0.154659,0.142857,0.366667,0.400000,0.666667,0.123304,0.492537,0.529412,...,0.782609,0.545455,0.107143,0.022743,0.138015,0.715495,0.0,0.0,45.653680,-122.546300
3738,0.001072,0.000000,0.081793,0.093596,0.500000,0.300000,0.500000,0.087527,0.417910,0.422460,...,0.826087,0.575758,0.107143,0.015797,0.114150,0.549791,0.0,0.0,36.850769,-76.285873
3739,0.006971,0.470608,0.200055,0.389163,0.162500,0.150000,0.725000,0.307925,0.421642,0.219251,...,0.739130,0.333333,0.321429,0.031257,0.092131,0.695028,2.0,1.0,34.075380,-84.294090


## Split Model Data by Cluster

In [14]:
y_dev = y_dev.reset_index()['priv_pay_median']
y_test = y_test.reset_index()['priv_pay_median']

X_dev_list = []
y_dev_list = []
X_test_list = []
y_test_list = []

for cluster_label in model_data["cluster"].unique():
    X_dev_list.append(X_dev_imputed[X_dev_imputed["cluster"] == cluster_label])
    y_dev_list.append(y_dev[X_dev_imputed["cluster"] == cluster_label])
    X_test_list.append(X_test_imputed[X_test_imputed["cluster"] == cluster_label])
    y_test_list.append(y_test[X_test_imputed["cluster"] == cluster_label])
    
    print(X_dev_list[-1].shape[0] / X_test_list[-1].shape[0])

4.116788321167883
3.949404761904762
3.0384615384615383


## Run XGBoost model

In [15]:
train_mapes = []
train_sizes = []
test_mapes = []
test_sizes = []

# Train test split
for idx in range(0,len(X_dev_list)):

    X_train, X_val, y_train, y_val = train_test_split(X_dev_list[idx],
                                                        y_dev_list[idx],
                                                        train_size = TRAIN_TEST_PROPORTION,
                                                        random_state = RDM_SEED)
    
    # Parameterization
    mono = {'site': 1}

    param_grid = {
#         'n_estimators':[100, 150, 200],
#         'max_depth':[5,10,15],
#         'learning_rate': [0.1,0.2,0.3],
#         'gamma':[0, 0.01,0.1],
#         'min_child_weight':[0,1],
#         'reg_lambda':[0,0.25,0.5,0.75]
    }
    
    # Create, run, and tune (if applicable) model
    xgb_param_tuning_model = xgb.XGBRegressor(monotone_constraints = mono,
                                              n_estimators = 200,
                                              max_depth = 10,
                                              gamma = 0,
                                              min_child_weight = 0,
                                              learning_rate = 0.1,
                                              reg_lambda = 0.75
                                             )
    
    xgb_mono_model = GridSearchCV(xgb_param_tuning_model, param_grid, scoring='neg_mean_absolute_percentage_error')
    xgb_mono_model.fit(X_train, y_train)
    
    # Output optimal params (if applicable)
    print(f"Best parameters (if grid search was applied): {xgb_mono_model.best_params_}")
    
    # Predict on train and test data
    y_train_pred_xgb = xgb_mono_model.predict(X_train)
    y_test_pred_xgb = xgb_mono_model.predict(X_test_list[idx])

    # Store results
    train_sizes.append(len(X_train))
    test_sizes.append(len(X_test_list[idx]))
    train_mapes.append(mean_absolute_percentage_error(y_true=y_train, y_pred=y_train_pred_xgb))
    test_mapes.append(mean_absolute_percentage_error(y_true=y_test_list[idx], y_pred=y_test_pred_xgb))
    

train_mapes = np.array(train_mapes)
train_sizes = np.array(train_sizes)
test_mapes = np.array(test_mapes)
test_sizes = np.array(test_sizes)

# Output results?
print(f"Monotonic XGBoost with Threshold >{COUNT_THRESH} claims for training set:")
print(f"Train MAPEs: {train_mapes}")
print(f"Train sizes: {train_sizes}")
print(f"Test MAPEs: {test_mapes}")
print(f"Test sizes: {test_sizes}")
print(f"Total train MAPE: {((train_mapes * train_sizes) / (train_sizes.sum())).sum()}")
print(f"Total test MAPE: {((test_mapes * test_sizes) / (test_sizes.sum())).sum()}")

C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

Best parameters (if grid search was applied): {}


C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

Best parameters (if grid search was applied): {}


C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\rogmo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future

Best parameters (if grid search was applied): {}
Monotonic XGBoost with Threshold >34 claims for training set:
Train MAPEs: [3.26319822e-03 5.50654172e-04 9.79170897e-07]
Train sizes: [1804 1061  126]
Test MAPEs: [0.16071486 0.16507588 0.1751726 ]
Test sizes: [548 336  52]
Total train MAPE: 0.0021635496632870133
Total test MAPE: 0.16308356208630484
